In [28]:
from PyPDF2 import PdfReader

reader = PdfReader("data/mental.pdf")
text = ""
for page in reader.pages:
    text += page.extract_text()


In [29]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = splitter.split_text(text)


In [30]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

loader = PyPDFLoader("data/mental.pdf")
pages = loader.load_and_split()

# Check metadata
for page in pages[:3]:
    print(page.metadata) 

{'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-07-28T11:57:49+00:00', 'source': 'data/mental.pdf', 'total_pages': 204, 'page': 0, 'page_label': '1'}
{'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-07-28T11:57:49+00:00', 'source': 'data/mental.pdf', 'total_pages': 204, 'page': 1, 'page_label': '2'}
{'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-07-28T11:57:49+00:00', 'source': 'data/mental.pdf', 'total_pages': 204, 'page': 2, 'page_label': '3'}


In [31]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

from dotenv import load_dotenv
import os

load_dotenv()

embedding = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))
vectorstore = FAISS.from_documents(pages, embedding)
vectorstore.save_local("mental_health_index")

embedding_model = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))



In [32]:
retrieved_docs = vectorstore.similarity_search("How do I help someone having a panic attack?")

In [33]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

llm = OpenAI(
    temperature=0.5,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever())

response = qa_chain.run("What should I do if someone is having suicidal thoughts?")
print(response)


 If someone is having suicidal thoughts, it is important to take them seriously and seek professional help. Offer to go with them to a mental health professional or call a suicide hotline for guidance. Do not dismiss their thoughts or make jokes about suicide. It is important to provide support and take their safety seriously.
